## Predicting Temperature of Nuclear Waste Canisters ##

This file contains the code to run, with a few comments. For more details please refer to the logs.ipynb file.

### **Imports** 

In [2]:
import numpy as np
import pandas as pd

### **Loading of the datasets** 

In [3]:
coordinates_test = pd.read_csv("data/Coordinates_Test.csv")
coordinates_train = pd.read_csv("data/Coordinates_Training.csv")
humidity_test = pd.read_csv("data/Test_Time_humidity.csv")
pressure_test = pd.read_csv("data/Test_Time_pressure.csv")
humidity_train = pd.read_csv("data/Training_data_humidity.csv")
pressure_train = pd.read_csv("data/Training_data_pressure.csv")
temperature_train = pd.read_csv("data/Training_data_temperature.csv")

### **Visualizing the datasets**

### **Feature engineering**